<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/243_PredRevenue_Gap_Orchestrator_Tier2_BizLogic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gap Detection Utilities for Revenue Gap Orchestrator

In [ ]:

"""
Gap Detection Utilities for Revenue Gap Orchestrator

Detect revenue gaps, churn risks, and below-baseline performance.
"""

from typing import Dict, List, Any, Optional


def detect_declining_revenue_gap(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    threshold: float = -15.0
) -> Optional[Dict[str, Any]]:
    """
    Detect if customer has declining revenue gap.

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline data (from revenue analysis)
        threshold: Percentage decline threshold (e.g., -15.0 for 15% decline)

    Returns:
        Gap dictionary if detected, None otherwise
    """
    trend_percentage = revenue_baseline.get("trend_percentage", 0.0)
    revenue_trend = revenue_baseline.get("revenue_trend", "stable")

    # Check if declining and below threshold
    if revenue_trend == "declining" and trend_percentage <= threshold:
        recent_avg = revenue_baseline.get("recent_weeks_avg", 0.0)
        baseline_avg = revenue_baseline.get("baseline_weeks_avg", 0.0)
        gap_amount = recent_avg - baseline_avg

        return {
            "customer_id": customer_id,
            "gap_type": "declining_revenue",
            "current_revenue": recent_avg,
            "expected_revenue": baseline_avg,
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": round(trend_percentage, 2),
            "severity": _determine_severity(trend_percentage),
            "weeks_at_risk": 0,  # Will be calculated separately
            "rationale": f"Customer revenue declined {abs(trend_percentage):.1f}% from baseline"
        }

    return None




# 🧠 **Deep Dive: `detect_declining_revenue_gap()`**

This function answers a *very specific* business question:

> **“Is this customer’s revenue dropping so much that we need to take action?”**

Gap detection is ALWAYS tied to *actionability*, not prediction.
This function is the **trigger** for one specific type of action:
📉 **Declining Revenue Intervention**

Let’s go through it step by step.

---

# 🔍 **1. Inputs**

```python
def detect_declining_revenue_gap(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    threshold: float = -15.0
)
```

### You pass in:

1. **customer_id** — who we’re analyzing
2. **revenue_baseline** — the output of revenue analysis

   * includes trend %, baseline avg, recent avg
3. **threshold** — how sensitive the detection is

   * default: -15% decline

### WHY this matters:

* Every detector is designed to be **configurable**
* Business teams can tune thresholds WITHOUT changing code
* The orchestrator is built for **non-engineer operators** to adjust rules

---

# 🔍 **2. Extract trend data**

```python
trend_percentage = revenue_baseline.get("trend_percentage", 0.0)
revenue_trend = revenue_baseline.get("revenue_trend", "stable")
```

This data was computed earlier by the revenue analysis utilities.

Example:

* baseline avg = \$100
* recent avg = \$70
* trend_percentage = -30%
* revenue_trend = "declining"

The logic here separates:

* **numeric signal** (`trend_percentage`)
* **categorical label** (`revenue_trend`)

This is important because:

* Rules can trigger on *labels* (“declining”)
* Other systems may trigger on *numbers* (-30%)

This dual representation helps make the system flexible.

---

# 🔍 **3. The core detection logic**

```python
if revenue_trend == "declining" and trend_percentage <= threshold:
```

You can read this in English as:

> *“If the customer is declining AND that decline is worse than the allowed threshold…”*

Example with threshold -15%:

* decline = -30% → trigger
* decline = -12% → no trigger

This rule:

* prevents false positives
* keeps responses focused
* ensures actionability

### ✨ THIS IS THE HEART OF GAP DETECTION.

Gap detection **always** follows the same pattern:

> Check a condition → if true → return a structured trigger event.

---

# 🔍 **4. Compute the gap**

```python
recent_avg = revenue_baseline.get("recent_weeks_avg", 0.0)
baseline_avg = revenue_baseline.get("baseline_weeks_avg", 0.0)
gap_amount = recent_avg - baseline_avg
```

This converts a **percentage decline** into a **dollar value gap**.

Example:

* baseline = \$100
* recent = \$70
* gap_amount = 70 - 100 = -30

Why convert to dollars?
Because action requires business impact.

Stakeholders don’t react to:

* “Decline of -30%”

They react to:

* “This customer is down \$30 per week, or \$120 per month.”

This ties analytics → action → revenue.

---

# 🔍 **5. Build the gap event object**

```python
return {
    "customer_id": ...,
    "gap_type": "declining_revenue",
    "current_revenue": recent_avg,
    "expected_revenue": baseline_avg,
    "gap_amount": round(gap_amount, 2),
    "gap_percentage": round(trend_percentage, 2),
    "severity": _determine_severity(trend_percentage),
    "weeks_at_risk": 0,
    "rationale": f"..."
}
```

This dictionary is **the standardized “event” object** that flows into:

* scoring → ranking → action recommendations → reporting

Every gap detector returns a similar object.

This is extremely important because:

## ⭐ You are designing an event-driven reasoning system.

Just like:

* fraud alerts
* anomaly detection
* predictive maintenance

Your orchestrator transforms raw data → meaningful events.

---

# 🔍 **6. Key concepts this teaches you**

## **1. Business Rules → Computed Triggers**

This function turns business intuition:

> “This customer is declining too much”

into deterministic logic:

> If trend < -15% → return a declining revenue event

This is the foundation of enterprise decision automation.

---

## **2. Deterministic Rules Complement ML**

Notice:

* No ML model is used here
* ML *could* influence the signals feeding this rule
  But the rule **decides** actionability.

This is the core philosophy of best-in-class orchestrators:

> **ML predicts.
> Business rules decide when to act.**

---

## **3. Clear separation of responsibilities**

Revenue analysis provides:

* trend
* averages
* baselines

Gap detection transforms those signals into:

* decisions
* triggers
* alerts

This separation makes the system:

* modular
* testable
* explainable (critical for enterprise use)
* easy to extend (just add new detectors)

---

## **4. Structured, explainable output**

The return object ALWAYS includes:

* type
* severity
* rationale
* numeric impact

This makes LLMs and UIs incredibly easy to build on top.

---

## **5. Parameterized thresholds**

The system is built to allow business teams to change:

* sensitivity
* thresholds
* definitions

THAT is the hallmark of production-ready automation systems.

---

# 🔥 TL;DR — What you should be learning here

### ✔ How to write clean, explainable business rules

### ✔ How orchestrators convert analysis → events → actions

### ✔ How rule-based triggers complement ML

### ✔ How to structure "gap events" for downstream scoring

### ✔ How thresholds & severity levels create business-friendly logic

### ✔ How to design modular detectors that are easy to expand

This is foundational knowledge for:

* AI Agents
* Decision Engines
* Business Logic Systems
* Automated Reasoning
* Event-Based Workflows
* Predictive Analytics Pipelines




# Detect Below Baseline Gap

In [ ]:
def detect_below_baseline_gap(
    customer_id: str,
    revenue_baseline: Dict[str, Any],
    current_revenue: float,
    threshold: float = -20.0
) -> Optional[Dict[str, Any]]:
    """
    Detect if customer is below baseline by threshold.

    Args:
        customer_id: Customer ID
        revenue_baseline: Revenue baseline data
        current_revenue: Current period revenue (e.g., recent weeks average)
        threshold: Percentage below baseline threshold (e.g., -20.0 for 20% below)

    Returns:
        Gap dictionary if detected, None otherwise
    """
    baseline_avg = revenue_baseline.get("baseline_weeks_avg", 0.0)

    if baseline_avg <= 0:
        return None

    # Calculate percentage difference
    percentage_diff = ((current_revenue - baseline_avg) / baseline_avg) * 100

    # Check if below threshold
    if percentage_diff <= threshold:
        gap_amount = current_revenue - baseline_avg

        return {
            "customer_id": customer_id,
            "gap_type": "below_baseline",
            "current_revenue": round(current_revenue, 2),
            "expected_revenue": round(baseline_avg, 2),
            "gap_amount": round(gap_amount, 2),
            "gap_percentage": round(percentage_diff, 2),
            "severity": _determine_severity(percentage_diff),
            "weeks_at_risk": 0,  # Will be calculated separately
            "rationale": f"Customer revenue {abs(percentage_diff):.1f}% below baseline"
        }

    return None



# 🔍 **What `detect_below_baseline_gap` Does — In One Paragraph**

This function checks whether a customer’s **current revenue** (usually their recent 4-week average) has fallen **below their baseline revenue** (their first 4-week average) by more than a specified percentage threshold (default: **-20%**). If the customer is spending significantly less than expected, the function returns a **gap object** describing the severity, the amount of the drop, and the rationale. If their current spend is *not* low enough to trigger the rule, it returns `None`.

---

# 🧩 **Key Logic in 3 Steps**

### **1️⃣ Skip customers with no baseline**

If baseline average ≤ 0, return `None`.

### **2️⃣ Compute drop %**

```
percentage_diff = ((current - baseline) / baseline) * 100
```

### **3️⃣ If drop exceeds threshold (e.g., -20%)**

Return a structured gap:

* `gap_type = "below_baseline"`
* Includes gap amount, percentage, severity, rationale.

Otherwise → no gap.

---

# 🧠 **Why this matters**

This rule catches customers who aren’t *declining* over time necessarily, but whose **current performance is simply below where it should be**.

It’s one of the simplest but most important gap detectors.




# Detect Churn Risk

In [ ]:
def detect_churn_risk(
    customer_id: str,
    sales_records: List[Dict[str, Any]],
    zero_weeks_threshold: int = 2
) -> Optional[Dict[str, Any]]:
    """
    Detect churn risk based on consecutive zero spend weeks.

    Args:
        customer_id: Customer ID
        sales_records: List of sales records (sorted by date, most recent last)
        zero_weeks_threshold: Number of consecutive zero spend weeks to flag

    Returns:
        Churn risk dictionary if detected, None otherwise
    """
    if not sales_records:
        return None

    # Sort by date to ensure correct order
    sorted_records = sorted(
        sales_records,
        key=lambda x: x.get('week_start_date', '')
    )

    # Count consecutive zero spend weeks from the end (most recent)
    consecutive_zeros = 0
    for record in reversed(sorted_records):
        weekly_spend = record.get('weekly_spend', 0.0)
        if weekly_spend == 0.0:
            consecutive_zeros += 1
        else:
            break

    # Check if threshold met
    if consecutive_zeros >= zero_weeks_threshold:
        # Calculate churn risk score (0-1)
        # More consecutive zeros = higher risk
        risk_score = min(1.0, consecutive_zeros / 4.0)  # Max risk at 4+ weeks

        risk_factors = []
        if consecutive_zeros >= zero_weeks_threshold:
            risk_factors.append("zero_spend_weeks")

        # Check if declining trend
        if len(sorted_records) >= 4:
            recent_avg = sum(r.get('weekly_spend', 0.0) for r in sorted_records[-4:]) / 4
            baseline_avg = sum(r.get('weekly_spend', 0.0) for r in sorted_records[:4]) / 4
            if recent_avg < baseline_avg * 0.7:  # 30% decline
                risk_factors.append("declining_trend")

        return {
            "customer_id": customer_id,
            "churn_risk_score": round(risk_score, 2),
            "risk_factors": risk_factors,
            "weeks_since_last_purchase": consecutive_zeros,
            "predicted_churn_probability": round(risk_score * 0.9, 2)  # Slightly conservative
        }

    return None



# 🔍 **Deep Dive: `detect_churn_risk()`**

This is one of the *most important* utilities in the entire orchestrator because:

* It detects the earliest and strongest signal of customer churn
* It is simple, rule-based, deterministic, and explainable
* It is designed to be easily extended with ML in the future
* It fits perfectly into the “Explain → Detect → Trigger” pattern

Let’s break it down.

---

# 1️⃣ **Input → What the function receives**

It accepts:

* `customer_id` → the customer being evaluated
* `sales_records` → their weekly purchase history
* `zero_weeks_threshold` → how many zero-spend weeks before being flagged (default = **2**)

This is intentionally light: no heavy computations, no external dependencies.

---

# 2️⃣ **Sorting Sales Records**

```python
sorted_records = sorted(sales_records, key=lambda x: x.get('week_start_date', ''))
```

✔ Ensures chronological order
✔ Critical for “counting backward”
✔ Makes the logic stable and predictable

---

# 3️⃣ **Core Churn Logic: Count Consecutive Zero-Spend Weeks**

```python
consecutive_zeros = 0
for record in reversed(sorted_records):
    weekly_spend = record.get('weekly_spend', 0.0)
    if weekly_spend == 0.0:
        consecutive_zeros += 1
    else:
        break
```

What this does:

* Start at **most recent week**
* Count how many weeks the customer spent **$0**
* Stop counting when spending resumes

### 🧠 Why this works (business logic perspective):

In nearly every subscription, retail, or loyalty industry:

> **Customers who stop spending for multiple periods in a row are at high churn risk.**

This is the simplest, clearest, most explainable churn signal.

---

# 4️⃣ **Threshold Check**

```python
if consecutive_zeros >= zero_weeks_threshold:
```

Default threshold:

* 1 zero week → normal
* **2 zero weeks → churn warning**
* 3+ zero weeks → high risk
* 4+ zero weeks → extreme risk

This rule is:

* simple
* configurable
* easy for executives to understand
* grounded in consumer behavior

---

# 5️⃣ **Calculate a Risk Score**

```python
risk_score = min(1.0, consecutive_zeros / 4.0)
```

This maps 0–4+ zero weeks to a churn risk of 0–1.

Examples:

| Zero weeks | Risk Score |
| ---------- | ---------- |
| 1          | 0.25       |
| 2          | 0.50       |
| 3          | 0.75       |
| 4+         | 1.00       |

🧠 *Nice part:*
This creates a **continuous risk signal**, not a binary yes/no.

---

# 6️⃣ **Identify Risk Factors**

```python
risk_factors.append("zero_spend_weeks")
```

Then it checks for a declining trend:

```python
if recent_avg < baseline_avg * 0.7:
    risk_factors.append("declining_trend")
```

This means the function is doing more than simply detecting churn:

> It explains *why* the customer is at risk.

This makes the output actionable, transparent, and perfect for a report.

---

# 7️⃣ **Return a Data-Rich Churn Object**

The function outputs:

```python
{
  "customer_id": ...,
  "churn_risk_score": ...,       # 0 - 1.0
  "risk_factors": [...],         # list of drivers
  "weeks_since_last_purchase": n,
  "predicted_churn_probability": ...  # risk_score * 0.9
}
```

You get:

* **Score**
* **Factors**
* **Time since last purchase**
* **Probability estimate**

And this object plugs directly into:

* the scoring pipeline
* the ranking algorithm
* the report generator

---

# 🧠 Business Takeaways

### **1. The function detects real churn, not statistical noise.**

Churn is a *behavioral event* not a prediction:
→ When a customer isn’t buying, they’re signaling exit.

### **2. Highly explainable and defensible**

Executives can intuitively understand:

* “They haven’t bought anything in 3 weeks”
* “Their spend dropped 30%”
* “This is why we’re flagging them”

### **3. Easily tuned per business**

* A grocery store might use 2 weeks
* A B2B SaaS might use 1 month
* A fashion store might use 6 weeks

This design supports configuration.

### **4. Perfect foundation for ML add-on**

ML could improve:

* churn probability estimation
* dynamic thresholds
* multi-signal fusion

But humans still love this simple rule.

---

# 🧠 Agent Architecture Takeaways

### **1. Outputs a standard “risk signal” object**

This object feeds directly into:

* scoring
* ranking
* reporting
* business actions

This is “signal → scoring → prioritization”.

### **2. Zero hallucination**

Everything is deterministic and grounded in customer history.

### **3. No model drift**

Unlike ML, rules don’t degrade over time unless business changes.

### **4. Modular and replaceable**

You can swap this function with an ML churn model later.

---

# 🎯 TL;DR Summary

`detect_churn_risk()`:

* Sorts customer records
* Counts recent zero-spend weeks
* Checks if it crosses a churn threshold
* Adds secondary evidence (declining trend)
* Counts risk factors
* Computes a churn score
* Returns a full, structured churn risk signal

It’s simple, powerful, reliable, and fully explainable.


